##1. Install the required code libraries

In [1]:
!pip install git+https://github.com/openai/whisper.git 
!sudo apt update && sudo apt install ffmpeg
!pip install librosa



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-p1ryy9cj
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-p1ryy9cj
  Resolved https://github.com/openai/whisper.git to commit c09a7ae299c4c34c5839a76380ae407e7d785914
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 61.2 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=798038 sha256=f431f3f0c5590e9c288b3f5e336f594ecc2636eb7e70ad4fea5fa9e9658accf7
  Stored in directory: /tmp/pip-ephem-wheel-cache-hkxcvs47/wheels/fe/03/29/e7919208d11b4ab32972cb448bb84a9a675d92cd52c9a48341
Successfully built openai-whisper
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-

100%|████████████████████████████████████████| 461M/461M [00:04<00:00, 118MiB/s]


In [2]:
import whisper
import time
import librosa
import soundfile as sf
import re
import os

# model = whisper.load_model("tiny.en")
# model = whisper.load_model("base.en")   
model = whisper.load_model("small.en") # load the small model
# model = whisper.load_model("medium.en")
# model = whisper.load_model("large")

In [19]:
pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


##2. Allow access to your Google Drive and add new folders

In [3]:

from google.colab import drive
drive.mount("/content/drive", force_remount=True)


folders =  ["WhisperAudio/", "WhisperAudio/ProcessedAudio/", "WhisperAudio/TextFiles/"]
for folder in folders:
  path = "/content/drive/MyDrive/" + folder
  if not os.path.exists(path): 
    os.mkdir(path)

Mounted at /content/drive


##3. Upload any audio files you want transcribed in the "WhisperAudio" folder in your Google Drive.

##4. Let the application search for new files and transcribe the audio files and save them to your Google Drive

In [15]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True) 


audio_folder = "/content/drive/MyDrive/WhisperAudio/"


import os
audio_files = []
audio_names = []
for file in os.listdir(audio_folder):
  if file.endswith(".m4a") or file.endswith(".mp3"):
    audio_files.append(audio_folder + file)
    audio_names.append(file)

for f in audio_files:    
  print(f)

if len(audio_files) == 0:
  print("You have no files.")

for i, file in enumerate(audio_files): 
  print(f"Processing {audio_names[i]}...")
  audio, sr = librosa.load(file, sr=16000, mono=True)
  pauses = librosa.effects.split(audio, top_db=30, frame_length=2048, hop_length=128)
  transcription = ""
  for start, end in pauses: 
    segment = audio[start:end]
    temp_file = "temp.wav"
    sf.write(temp_file, segment, sr, subtype='PCM_16')
    if os.path.getsize(temp_file) > 10000:

      result = model.transcribe(temp_file)
      text = result["text"]
      print(len(transcription.split(" ")), "words processed")
      transcription += text.strip() + " "
      os.remove(temp_file)
  print(f"Transcription of {audio_names[i]}:\n")
  print(transcription)
  print("\n")
 
  transcription = re.sub(r"\s\s+", "\n\n", transcription) 
  text_file = audio_folder + "/TextFiles/" + audio_names[i][:-4] + ".txt" 
  with open(text_file, "w") as f: 
    f.write(transcription)
  print(f"Saved transcription as {text_file}")

import shutil
processed_folder = "/content/drive/MyDrive/WhisperAudio/ProcessedAudio/"
if not os.path.exists(processed_folder):
  os.mkdir(processed_folder)
for file in audio_files: 
  shutil.move(file, processed_folder + os.path.basename(file))
  print(f"Moved {file} to {processed_folder}")

Mounted at /content/drive
/content/drive/MyDrive/WhisperAudio/SlL3KVEYak0.m4a
Processing SlL3KVEYak0.m4a...


<ipython-input-15-2c992764149f>:24: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, sr = librosa.load(file, sr=16000, mono=True)
/usr/local/lib/python3.9/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


1 words processed
Transcription of SlL3KVEYak0.m4a:

Are you trying to learn the language, but you run out of words to say? You can't speak with confidence, and you just don't have the opportunity to practice? All language learners dream of speaking the language, and they all run into these problems when learning alone. But what if you could learn a practical conversation in one lesson, where teachers slow down and explain every word in grammar rule, and start speaking in minutes? Imagine how much you'd speak after five lessons, 10 lessons, 20 lessons. Well, that's exactly how you learn with our step-by-step learning system. How does it work? If you get up to 35% off with the Ready, Set, Speak sale going on now, you unlock bite-sized conversation lessons by real teachers. Each is just minutes long, so you can learn fast. Just press play on a lesson, learn a new conversation, and our teachers get you speaking in minutes. You get these conversations slowed down, translated, and broken do

In [25]:
!pip install gensim==3.8.3


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.4/23.4 MB 29.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gensim: filename=gensim-3.8.3-cp39-cp39-linux_x86_64.whl size=26528086 sha256=af7f42249c8edcb5c5b6b088ab83f3515fb3ac185a30dc84a8bd7bded2eff3b9
  Stored in directory: /root/.cache/pip/wheels/ca/5d/af/618594ec2f28608c1d6ee7d2b7e95a3e9b06551e3b80a491d6
Successfully built gensim
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.1
    Uninstalling gensim-4.3.1:
      Successfully uninstalled gensim-4.3.1


In [13]:
import argostranslate.package
import argostranslate.translate

from_code = "en"
to_code = "ar"

# Download and install Argos Translate package
argostranslate.package.update_package_index()
available_packages = argostranslate.package.get_available_packages()
package_to_install = next(
    filter(
        lambda x: x.from_code == from_code and x.to_code == to_code, available_packages
    )
)
argostranslate.package.install_from_path(package_to_install.download())

# Translate
translatedText = argostranslate.translate.translate(transcription, from_code, to_code)
print(translatedText)
# '¡Hola Mundo!'

هل تحاول أن تتعلم اللغة، لكنّك نفدت الكلمات لتقولها؟ لا يمكنك التحدث بثقة، وأنت فقط ليس لديك الفرصة لممارسة؟ جميع المتعلمين اللغويين يحلمون بالحديث عن اللغة وكلهم يصادفون هذه المشاكل عندما يتعلمون وحدهم لكن ماذا لو تعلمت محادثة عملية في درس واحد حيث يبطئ المعلمون ويشرحون كل كلمة في قاعدة "غراممار" ويبدأون بالتحدث في دقائق؟ تخيل كم ستتكلم بعد خمسة دروس، 10 دروس، 20 درساً حسناً، هذه هي الطريقة التي تتعلم بها نظامنا التعليمي المتدرج كيف يعمل؟ إذا حصلت على ما يصل إلى 35٪ قبالة مع استعداد، مجموعة، والحديث عن بيع يجري الآن، كنت فتح باب دروس في المحادثة ذات الحجم العض من قبل معلمين حقيقيين. كل واحد فقط دقائق طويلة، حتى تتمكن من التعلم بسرعة. فقط اضغطي على الدرس وتعلمي محادثة جديدة و معلمينا يجعلونك تتكلمين في دقائق تَحْصلُ على هذه المحادثاتِ تَباطأَ، تَترجّمَ، وتَحطّمَ خطّاً، لذا تَفْهمُ كُلّ شيءَ. يُمكنك أن تُغرس نفسك في مسارات المحادثة يمكنك التدرب على التحدث ومقارنة نطقك مع السكان الأصليين يمكنك التعلم في أي مكان، في أي وقت، في المنزل، على القطار، مع تطبيقنا. تحصل على أدوات دراسية تساعدك ع

In [14]:
import gensim
from gensim.summarization import summarize

# text to be summarized
text = transcription

# summarize the text
summary = summarize(text, ratio=0.5)

# print the summary
print(summary)


Are you trying to learn the language, but you run out of words to say?
All language learners dream of speaking the language, and they all run into these problems when learning alone.
But what if you could learn a practical conversation in one lesson, where teachers slow down and explain every word in grammar rule, and start speaking in minutes?
If you get up to 35% off with the Ready, Set, Speak sale going on now, you unlock bite-sized conversation lessons by real teachers.
Each is just minutes long, so you can learn fast.
Just press play on a lesson, learn a new conversation, and our teachers get you speaking in minutes.
You get study tools that help you memorize words and conversations fast.
And if you want personalized learning, you can even get your own teacher.
So if you want to finally learn and speak the language, get up to 35% off our learning system with the Ready, Set, Speak sale.
